In [1]:
import sys 
import os
sys.path.append(os.path.abspath("/home/xilinx/jupyter_notebooks/thies/notebooks"))


from fpga_implementation import DESEncryptor
from pydes_implementation import PyDES

In [2]:
encryptor = DESEncryptor(max_parallel_send=2 ** 8)

# Example
message = "AAAAAAA" # 7 bytes, will be padded
key = "BBBBBBBB"     # 8 bytes
encrypted_result = encryptor.encrypt(message, key)
print(f"Encrypted result: {encrypted_result}")

message_long = "AAAAAAAAAAAAAAA" # 15 bytes
encrypted_result_long = encryptor.encrypt(message_long, key)
print(f"Encrypted result (long): {encrypted_result_long}")


FileNotFoundError: [Errno 2] No such file or directory: '/home/xilinx/jupyter_notebooks/thies/notebooks/DMA/des_decrypt_bd_wrapper.bit'

In [ ]:
# Test Cases
def test_compare_des_encrypt():
    """
    Test to compare the output of the PYNQ DES implementation with the pyDes library.
    """
    fpga_encryptor = DESEncryptor() 
    pyDes_encryptor = PyDES()

    test_vectors = [
        {"message": "ABCDEFGH", "key": "12345678"},
        {"message": "ABCDEFGHabcdefgh", "key": "12345678"},
        {"message": "ABCDEFG", "key": "12345678"},
        {"message": "", "key": "12345678"},
        {"message": "ABCDEFGHIJKLMNOPQRSTUVWXYZ123456", "key": "12345678"},
    ]

    for test_vector in test_vectors:
        message = test_vector["message"]
        key = test_vector["key"]
        print(f"Message: {message}, Key: {key}")

        # Get results from both implementations
        pynq_result_bytes = fpga_encryptor.encrypt(message, key)
        print(f"PYNQ Result (bytes): {pynq_result_bytes.hex()}")

        pyDes_result_bytes = pyDes_encryptor.encrypt(message, key)
        print(f"pyDes Result (bytes): {pyDes_result_bytes.hex()}")

        # Compare the results.  Note that the hex representation should be the same.
        assert pynq_result_bytes == pyDes_result_bytes, f"Results do not match for message: {message}, key: {key}"

test_compare_des_encrypt()


In [ ]:
# Test Cases
def test_compare_des_decrypt():
    """
    Test to compare the output of the PYNQ DES implementation with the pyDes library.
    """
    fpga_encryptor = DESEncryptor() 
    pyDes_encryptor = PyDES()

    test_vectors = [
        {"message": "ABCDEFGH", "key": "12345678"},
        {"message": "ABCDEFGHabcdefgh", "key": "12345678"},
        {"message": "ABCDEFG", "key": "12345678"},
        {"message": "", "key": "12345678"},
        {"message": "ABCDEFGHIJKLMNOPQRSTUVWXYZ123456", "key": "12345678"},
    ]

    for test_vector in test_vectors:
        message = test_vector["message"]
        key = test_vector["key"]
        print(f"Message: {message}, Key: {key}")

        # Get results from both implementations
        pynq_result_bytes = fpga_encryptor.decrypt(message, key)
        print(f"PYNQ Result (bytes): {pynq_result_bytes.hex()}")

        pyDes_result_bytes = pyDes_encryptor.decrypt(message, key)
        print(f"pyDes Result (bytes): {pyDes_result_bytes.hex()}")

        # Compare the results.  Note that the hex representation should be the same.
        assert pynq_result_bytes == pyDes_result_bytes, f"Results do not match for message: {message}, key: {key}"
        

test_compare_des_decrypt()


In [ ]:
# Performance

def compare_des_timing():
    """
    Compares the execution time of the PYNQ DES implementation with the pyDes library.
    """

    test_vectors = [
        {"message": "ABCDEFGH", "key": "12345678"},
        {"message": "ABCDEFGHabcdefgh", "key": "12345678"},
        {"message": "ABCDEFG", "key": "12345678"},
        {"message": "", "key": "12345678"},
        {"message": "ABCDEFGHIJKLMNOPQRSTUVWXYZ123456", "key": "12345678"},
        {"message": "ABCDEFGHIJKLMNOPQRSTUVWXYZ123456ABCDEFGHIJKLMNOPQRSTUVWXYZ123456", "key": "12345678"},
        {"message": "A" * 128, "key": "12345678"},
        {"message": "A" * 128 * 4, "key": "12345678"},
#         {"message": "A" * 2 ** 20 * 8, "key": "12345678"},
    ]
    
    
    largest_size = max(map(lambda x: len(x["message"]), test_vectors)) // 8
    print("Instanting DES encryptor with size: ", largest_size)
    
    encryptor = DESEncryptor(logging=False, max_parallel_send=largest_size)  # Create an instance of DESEncryptor
    

    num_iterations = 10  # Number of times to repeat the encryption for timing

    print("Timing Comparison:")
    print(f"Number of iterations: {num_iterations}")

    for test_vector in test_vectors:
        message = test_vector["message"]
        key = test_vector["key"]

        # Time PYNQ encryption
        start_time_pynq = time.time()
        for _ in range(num_iterations):
            pynq_result_hex = encryptor.encrypt(message, key)
        end_time_pynq = time.time()
        pynq_time = end_time_pynq - start_time_pynq

        # Time pyDes encryption
        start_time_pydes = time.time()
        for _ in range(num_iterations):
            pyDes_result_bytes = encrypt_with_pyDes(message, key)
        end_time_pydes = time.time()
        pydes_time = end_time_pydes - start_time_pydes

        print(f"\nMessage: {message}, Key: {key}")
        print(f"PYNQ Time: {pynq_time:.6f} seconds")
        print(f"pyDes Time: {pydes_time:.6f} seconds")
        print(f"Speed up of: {pydes_time / pynq_time:.6f}")

        # Compare the results (optional, for functional verification)
        pynq_result_bytes = bytes.fromhex(pynq_result_hex)
        pyDes_result_bytes = encrypt_with_pyDes(message, key)
        assert pynq_result_bytes == pyDes_result_bytes, f"Results do not match for message: {message}, key: {key}"

compare_des_timing()
print("Done")